# Browse TD-MPC2 runs (local + W&B)

Examples for working with `discover/runs.py` from a notebook. Run this notebook from the repo root or the `discover/` directory.

## Setup
- Update `logs_dir` if your logs live elsewhere.
- Set `do_wandb` to `True` and `wandb_project` to fetch remote runs.

In [2]:
from pathlib import Path
import pandas as pd

from discover.runs import discover_local_logs, discover_wandb_runs

logs_dir = Path('../tdmpc2/logs')
do_wandb = False  # flip to True to query W&B
wandb_project = 'wm-planning/mmbench'  # entity/project

ModuleNotFoundError: No module named 'pandas'

In [ ]:
local_df = discover_local_logs(logs_dir, limit=None)
print(f'Local runs: {len(local_df)}')
local_df.head()

In [ ]:
if do_wandb:
    wandb_df = discover_wandb_runs(wandb_project, limit=500, include_artifacts=False)
    print(f'W&B runs: {len(wandb_df)}')
else:
    wandb_df = pd.DataFrame()

dfs = [df for df in (local_df, wandb_df) if not df.empty]
df_all = pd.concat(dfs, ignore_index=True, sort=False) if dfs else pd.DataFrame()
print(f'Total rows combined: {len(df_all)}')
df_all.head()

## Max runtime per task
For each task, pick the run with the maximum `runtime` (assumed to live under `summary['runtime']`). If a task has no runtime recorded, it is included with `runtime = NaN`.

In [ ]:
if df_all.empty:
    print('No runs found; populate logs_dir or enable do_wandb.')
else:
    def extract_runtime(row):
        summary = row.get('summary')
        if isinstance(summary, dict):
            return summary.get('runtime')
        return None

    df_all['runtime'] = pd.to_numeric(df_all.apply(extract_runtime, axis=1), errors='coerce')
    # idx of max runtime per task (fill NaN with -1 to keep the first row if all NaN)
    idx = (
        df_all.assign(runtime_filled=df_all['runtime'].fillna(-1))
        .groupby('task', dropna=False)['runtime_filled']
        .idxmax()
    )
    best_by_task = df_all.loc[idx].copy()
    best_by_task = best_by_task.sort_values('task')
    display(best_by_task[[
        'task',
        'runtime',
        'exp_name',
        'seed',
        'source',
        'ckpt_path',
        'url',
        'run_dir',
        'summary',
    ]])
